In [ ]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

import statsmodels as sm

import matplotlib.pyplot as plt
%matplotlib inline

from stldecompose import decompose, forecast
from stldecompose.forecast_funcs import (naive,
                                         drift, 
                                         mean, 
                                         seasonal_naive)
data = pd.read_csv('Data/data_time_series.csv', sep = ',')
data['Period'] = pd.to_datetime(data['Period'])

## 1. Интерполяция (сглаживание) [Первый этап]

In [ ]:
# 1. Just Approximation:
import numpy as np
from datetime import datetime

dta = data

dta['Period'] = dta['Period'].apply(lambda x: datetime.strptime(''.join(str(x).split(' ')[:-1]), '%Y-%m-%d'))


plt.figure(figsize=(15,7))
plt.plot(dta['Period'].values.astype('datetime64[D]'), dta['y'].values)


n_data = len(dta['Period'].values.astype('datetime64[D]'))

# y = x^3*a3 + x^2*a2 + x*a1 + a0
model = np.polyfit(x = np.arange(n_data), y = np.array(dta['y']), deg = min_deg)
polynom = np.poly1d(model)

plt.plot(dta['Period'].values.astype('datetime64[D]'), polynom( np.arange(n_data) ), 'b--')
# x = [x1, x2, x3, x4, x5] = [1,2,3,4,5]
# y1 = x1^3*a3 + x1^2*a2 + x1*a1 + a0
# y2 = x2^3*a3 + x2^2*a2 + x3*a1 + a0
# y3 = x3^3*a3 + x3^2*a2 + x3*a1 + a0
# y = [y1, y2, y3, y4, y5]


from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import ml_metrics as metrics

print '\n\nR^2: %1.9f' % r2_score(dta['y'], polynom( np.arange(n_data) ))
print '\nRMSE (ml_metrics.metrics): %.9f' % metrics.rmse(dta['y'], polynom( np.arange(n_data) ))
print 'RMSE (scikit-learn): %.9f' % np.sqrt(mean_squared_error(dta['y'], polynom( np.arange(n_data) )))
print 'MAE (ml_metrics.metrics): %.9f' % metrics.mae(dta['y'], polynom( np.arange(n_data) ))
print 'MAE (scikit-learn): %.9f' % mean_absolute_error(dta['y'], polynom( np.arange(n_data) ))


In [ ]:
# https://stackoverflow.com/questions/47442102/how-to-find-the-best-degree-of-polynomials
import numpy as np
import matplotlib.pyplot as plt 

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from datetime import datetime

from sklearn.metrics import r2_score
import ml_metrics as metrics

dta = data
dta['Period'] = dta['Period'].apply(lambda x: datetime.strptime(''.join(str(x).split(' ')[:-1]), '%Y-%m-%d'))

n_data = len(dta['Period'].values.astype('datetime64[D]'))
X = np.arange(n_data).reshape(n_data, 1)
y = np.array(dta['y'])

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

rmses = []
maes = []
r2_scores = []
degrees = np.arange(1, 100)
min_rmse, min_mae, min_r2, min_deg, min_deg_mae, min_deg_r2 = 1e10, 1e10, 0.01, 0, 0, 0

for deg in degrees:

    # Train features
    poly_features = PolynomialFeatures(degree=deg, include_bias=False)
    x_poly_train = poly_features.fit_transform(x_train)

    # Linear regression
    poly_reg = LinearRegression()
    poly_reg.fit(x_poly_train, y_train)

    # Compare with test data
    x_poly_test = poly_features.fit_transform(x_test)
    poly_predict = poly_reg.predict(x_poly_test)
    
    # 1. RMSE:
    poly_mse = mean_squared_error(y_test, poly_predict) # Mean Square Error
    poly_rmse = np.sqrt(poly_mse) # Root Mean Square Root
    rmses.append(poly_rmse)
    
    # 2. MAE:
    poly_mae = mean_absolute_error(y_test, poly_predict)
    maes.append(poly_mae)
    
    # 3. R2 - score:
    poly_r2_score = r2_score(y_test.tolist(), poly_predict.tolist())
    r2_scores.append(poly_r2_score)

    # Cross-validation of degree
    if min_rmse > poly_rmse:
        min_rmse = poly_rmse
        min_deg = deg
    if min_mae > poly_mae:
        min_mae = poly_mae
        min_deg_mae = deg
    if min_r2 < poly_r2_score:
        min_r2 = poly_r2_score
        min_deg_r2 = deg

# Plot and present results
print('Best degree {} with RMSE {},\n\t\t {} with MAE {},\n\t\t {} with R2 {}'.format(min_deg, min_rmse, min_deg_mae, min_mae, min_deg_r2, min_r2))

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(degrees, rmses)
ax2 = ax.twinx()
ax.plot(degrees, maes, 'b-.')
ax2.plot(degrees, r2_scores, 'g--')
ax.set_yscale('log')
ax.set_xlabel('Degree')
ax2.set_ylabel('R2')
ax.set_ylabel('MAE')
ax.set_ylabel('RMSE')

In [ ]:
import matplotlib
matplotlib.colors
matplotlib.colors.rgb_to_hsv
matplotlib.colors.to_rgba
matplotlib.figure.Figure.get_size_inches
matplotlib.figure.Figure.subplots_adjust
matplotlib.axes.Axes.text
matplotlib.axes.Axes.hlines

dta = data
dta['Date'] = pd.to_datetime(dta['Period'])
dta = dta.set_index('Date')

# 1. Multiplicative 
# 1.1. Weekly
res_ymw = seasonal_decompose(dta['y'], model='multiplicative', freq = 7)
# 1.2. Monthly
res_ymm = seasonal_decompose(dta['y'], model='multiplicative', freq = 30)

# 2. Additive 
# 2.1. Weekly
res_yaw = seasonal_decompose(dta['y'], model='additive', freq = 7)
# 2.2. Monthly
res_yam = seasonal_decompose(dta['y'], model='additive', freq = 30)

# 3. STL-Lib Decompopsition by LOESS - Logistic Regression:
stl_y = decompose(dta[['y']], period=7)


# Just Aggregation:
# 1. Not Aggregated:
res_y = dta.y

# 2. Aggregated by Week (Moving Median):
res_ymedw = dta.y.resample('W').mean()

# 3. Aggregated by Month (Moving Median):
res_ymedm = dta.y.resample('M').mean()


# Plotting Charts:
plt.figure(figsize=(25,11))
plt.plot(res_ymw.trend, '--', label='Samara_Trend_Multi_7', color='darkgreen')
plt.plot(res_ymm.trend, '--', label='Samara_Trend_Multi_30', color='darkgreen')
plt.plot(res_yaw.trend, '--', label='Samara_Trend_Add_7', color='darkgreen')
plt.plot(res_yam.trend, '--', label='Samara_Trend_Add_30', color='darkgreen')
plt.plot(stl_y.trend, '-o', label='Samara_STL_Trend', color='darkgreen')

plt.plot(dta['y'], '-', label='Samara_Original', color='lime')
plt.plot(res_ymedw, 'g-.', label='Samara_Weekly_Megian')
plt.plot(res_ymedm, 'g-.', label='Samara_Monthy_Megian')
plt.plot(dta['Period'].values.astype('datetime64[D]'), polynom( np.arange(n_data) ), 'b-o')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

## 2. Экстраполяция aka Прогноз 

In [ ]:
polynom( np.arange(n_data) )

### Данные за вычетом тренда:

In [ ]:
# 1. ARIMA (для MA)
### Свойства временного ряда

# Коррелограмма:
# ARIMA(p of AR, q, d of MA)
trend = pd.DataFrame(data=(dta.y - polynom(np.arange(n_data)) ) )
y1diff = trend.diff(periods=1).dropna() # Первая разность ряда
# y1diff = np.array([trend[1] - trend[0], trend[2] - trend[1], ...])

# AdFuller: (Единичные корни)
# Original:
test = sm.tsa.stattools.adfuller(trend.iloc[:,0].values)
print 'ADF: ', test[0] 
print 'p-value: ', test[1]
print'Critical values: ', test[4]
if test[0]> test[4]['5%']: 
    print 'есть единичные корни, ряд не стационарен'
else:
    print 'единичных корней нет, ряд стационарен'
# Единичные корни: Первая разность ряда
test = sm.tsa.stattools.adfuller(y1diff.iloc[:,0].values)
print 'ADF: ', test[0]
print 'p-value: ', test[1]
print'Critical values: ', test[4]
if test[0]> test[4]['5%']: 
    print 'есть единичные корни, ряд не стационарен'
else:
    print 'единичных корней нет, ряд стационарен'
    
# y2diff = trend.diff(periods=2).dropna() # Вторая разность ряда   
# Единичные корни: Вторая разность ряда
test = sm.tsa.stattools.adfuller(y2diff.iloc[:,0].values)
print 'ADF: ', test[0]
print 'p-value: ', test[1]
print'Critical values: ', test[4]
if test[0]> test[4]['5%']: 
    print 'есть единичные корни, ряд не стационарен'
else:
    print 'единичных корней нет, ряд стационарен'

fig = plt.figure(figsize=(12,8))
plt.title('Ishodniy Ryad', loc='left')
ax1 = fig.add_subplot(211)
# ACF: p=25
fig = sm.api.graphics.tsa.plot_acf(trend_indexed.values.squeeze(), lags=8, ax=ax1)
# Take a look at Number of Lags!!!! lags=20, 25, 30, 100, ... etc.: coef_autocor = 0.75 at 21st
ax2 = fig.add_subplot(212)
# PACF: q=2
fig = sm.api.graphics.tsa.plot_pacf(trend_indexed, lags=8
                                    , ax=ax2)


In [ ]:
trend = pd.DataFrame(data=dta.y - polynom(np.arange(n_data)) )
trend

In [ ]:
res_ymedm

### Данные на тренде:

In [ ]:
# https://people.duke.edu/~rnau/411arim.htm#pdq
# https://onlinecourses.science.psu.edu/stat501/node/358/
from statsmodels.tsa.ar_model import AR
import statsmodels as sm

src_data_model = res_ymw.trend['2018-02-21':'2018-11-19'].apply(lambda x: int(x))

model = sm.tsa.arima_model.ARIMA(src_data_model, (1,0,1))
model = model.fit()
model.predict(start='2018-11-10', end='2018-11-25')

from sklearn.metrics import r2_score
import ml_metrics as metrics

pred = model.predict('2018-02-22', '2018-11-30')#, typ='levels')
trn = res_ymw.trend['2018-02-21':]
r2 = r2_score(trn[1:271], pred[1:271])
print 'R^2: %1.2f' % r2

print metrics.rmse(trn[1:271], pred[1:271])
print metrics.mae(trn[1:271], pred[1:271])

res_ymw.trend['2018-02-21':'2018-11-30'].apply(lambda x: int(x)).plot(figsize=(12,6))
#src_data_model.plot()
pred.plot(style='r--')


In [ ]:
# https://people.duke.edu/~rnau/411arim.htm#pdq
# https://onlinecourses.science.psu.edu/stat501/node/358/
from statsmodels.tsa.ar_model import AR
import statsmodels as sm

src_data_model = res_ymw.trend['2018-02-21':'2018-11-19'].apply(lambda x: int(x))

model = sm.tsa.arima_model.ARIMA(src_data_model, (1,0,5))
model = model.fit()
model.predict(start='2018-11-20', end='2018-11-25')

from sklearn.metrics import r2_score
import ml_metrics as metrics

pred = model.predict('2018-02-23', '2018-11-30')#, typ='levels')
trn = res_ymw.trend['2018-02-21':]
r2 = r2_score(trn[1:271], pred[1:271])
print 'R^2: %1.2f' % r2

print metrics.rmse(trn[1:271], pred[1:271])
print metrics.mae(trn[1:271], pred[1:271])

res_ymw.trend['2018-02-21':'2018-11-30'].apply(lambda x: int(x)).plot(figsize=(12,6))
#src_data_model.plot()
pred.plot(style='r--')

In [ ]:
# Plotting Charts:
plt.figure(figsize=(26,7))
plt.plot(res_ymedm, 'g-.', label='Samara_Monthy_Megian')



In [ ]:
trend.head()

In [ ]:
y1diff.head()

In [ ]:
y1diff_indexed = y1diff
y1diff_indexed.reset_index(inplace=True)
y1diff_indexed['Date'] = pd.to_datetime(y1diff_indexed['Date'])
y1diff_indexed = y1diff_indexed.set_index('Date')

In [ ]:
y1diff_indexed.head()